In [1]:
!pip install -q transformers torch ipywidgets flash-attn
!pip install -q accelerate>=0.26.0
!pip install -U -q bitsandbytes


In [2]:
import accelerate
print(accelerate.__version__)


1.1.1


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate
import bitsandbytes, flash_attn

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization. Use load_in_8bit=True for 8-bit.
    bnb_4bit_compute_dtype=torch.float16,  # Set computation to use FP16 for performance.
    bnb_4bit_use_double_quant=True,       # Optional: Enable double quantization for stability.
)

model_name = "NousResearch/Hermes-3-Llama-3.1-8B"
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16, 
    device_map="auto",
    quantization_config=quantization_config,
    attn_implementation="flash_attention_2"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
model.bnb_4bit_compute_dtype = torch.float16  # Align with `torch_dtype=torch.float16`


Checking memory

In [4]:
import torch

print(torch.cuda.memory_allocated() / (1024 ** 3), "GB allocated after loading")


5.6446614265441895 GB allocated after loading


In [5]:
from pydantic import BaseModel, ValidationError
import json

class StudentProfile(BaseModel):
    major: str
    bio: str

def validate_json(response: str):
    try:
        # Attempt to parse and validate the response as a StudentProfile
        student_profile = StudentProfile.parse_raw(response)
        return student_profile
    except ValidationError as e:
        # Catch invalid JSON or schema errors and return None
        print(f"Invalid JSON or structure: {e}")
        return None
    except json.JSONDecodeError as e:
        # Catch invalid JSON formatting errors
        print(f"JSON Decode Error: {e}")
        return None

In [6]:
chat_template = """<|im_start|>system
You are a helpful assistant that answers in JSON. Follow the exact JSON schema below:
<schema>
{{
    "major": "string",
    "bio": "string"
}}
</schema>
When writing the "bio", strictly adhere to these rules:
1. Only include the student's experiences, activities, achievements, projects, leadership positions, internships, research, and career goals.
2. Do NOT include any personal information such as the student's name, gender, school name, hometown, ethnicity, or other identifiable details.
3. Write the bio in 3-6 sentences, focusing on the professional and academic highlights of the student, using concise and formal language.
<|im_end|>
<|im_start|>user
Generate a college student profile for a {major} major. Write a "bio" that is detailed but does not include any personal identifiers, and only focuses on the student's academic and professional experiences, activities, and goals. <|im_end|>
<|im_start|>assistant
"""

major = "Computer Science"
formatted_prompt = chat_template.format(major=major)

In [7]:
print(formatted_prompt)

<|im_start|>system
You are a helpful assistant that answers in JSON. Follow the exact JSON schema below:
<schema>
{
    "major": "string",
    "bio": "string"
}
</schema>
When writing the "bio", strictly adhere to these rules:
1. Only include the student's experiences, activities, achievements, projects, leadership positions, internships, research, and career goals.
2. Do NOT include any personal information such as the student's name, gender, school name, hometown, ethnicity, or other identifiable details.
3. Write the bio in 3-6 sentences, focusing on the professional and academic highlights of the student, using concise and formal language.
<|im_end|>
<|im_start|>user
Generate a college student profile for a Computer Science major. Write a "bio" that is detailed but does not include any personal identifiers, and only focuses on the student's academic and professional experiences, activities, and goals. <|im_end|>
<|im_start|>assistant



In [8]:
import json
def generate_response(model, tokenizer, prompt, temperature=1.0, num_return_sequences=1, device = "cuda")->json:
    """
    Generate a response from the model given a prompt. Runs 4 times before failing and returning None.
    """
    
    for i in range(4):

        try:
            input_ids = tokenizer(prompt, padding=True, return_tensors="pt").input_ids.to(device)

            # Generate
            generated_ids = model.generate(
                input_ids,
                temperature=temperature, 
                repetition_penalty=1.1,
                num_return_sequences=num_return_sequences,
                do_sample=True, 
                max_length=512,
                eos_token_id=tokenizer.eos_token_id
            )
            
            # Decode and return the generated text
            response = tokenizer.decode(generated_ids[0][input_ids.shape[-1]:], skip_special_tokens=True, clean_up_tokenization_space=True)
            json_response = json.loads(response)
            return json_response
        
        except Exception as e:
            print(f"Attempt {i+1} failed with: {e}")
            print(response)
            if i == 3:
                return None

        

In [11]:
generate_response(model, tokenizer, formatted_prompt, temperature=0.5)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'major': 'Computer Science',
 'bio': "A dedicated and ambitious Computer Science student with a strong foundation in programming languages, data structures, and algorithms. Experienced in software development through various projects and internships. Actively involved in campus organizations focused on technology and innovation. Aspiring to contribute innovative solutions to industry challenges while pursuing a Master's degree and a career in software engineering."}

In [12]:
majors = {
    "STEM": [
        "Computer Science",
        "Mechanical Engineering",
        "Electrical Engineering",
        "Biology",
        "Nursing",
    ],
    "Arts": [
        "Graphic Design",
        "Music",
        "Theater",
    ],
    "Business": [
        "Business Administration",
        "Finance",
        "Marketing",
        "Accounting",
        "Economics"
    ],
    "Humanities": [
        "Psychology",
        "Communication",
        "Political Science",
        "Criminal Justice",
        "Education"
    ]
}

In [13]:
import pandas as pd

data = []
for field, majors in majors.items():

    for major in majors:
        formatted_prompt = chat_template.format(major=major)
        for _ in range(100):
            response = generate_response(model, tokenizer, formatted_prompt, temperature=0.9)
            if response:
                data.append({"field": field, "major": major, "bio": response["bio"]})
        print(f"FINISHED {major}\n\n")

df = pd.DataFrame(data)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Attempt 1 failed with: Expecting value: line 1 column 1 (char 0)
```json
{
    "major": "Computer Science",
    "bio": "A dedicated undergraduate pursuing a Bachelor's degree in Computer Science with a focus on software development and cybersecurity. Has completed coursework in data structures, algorithms, and programming languages including Java, Python, and C++. Gained valuable experience through internships at leading tech companies, working on project management and software quality assurance. Actively engaged in campus organizations focused on technology education and community outreach. Aspires to secure a position within the IT industry where they can utilize their technical skills and problem-solving abilities to create innovative solutions."
}
```


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Attempt 1 failed with: Expecting value: line 1 column 1 (char 0)
```json
{
    "major": "Computer Science",
    "bio": "A dedicated and skilled Computer Science student with experience in software development and database management. Has contributed to multiple group projects, demonstrating strong teamwork skills and effective communication abilities. Actively involved in campus organizations and community outreach programs, showcasing commitment to both academic excellence and social responsibility. Pursuing a Bachelor's degree with aspirations to secure employment within the technology industry, potentially specializing in areas such as cybersecurity or data analytics."
}
```


...

Attempt 1 failed with: Expecting value: line 1 column 1 (char 0)
```json
{
    "major": "Education",
    "bio": "A dedicated and enthusiastic Education major with extensive experience in teaching and mentorship roles. Significant involvement in local community outreach programs, where they developed and implemented engaging educational workshops for children aged 5-12. Actively volunteers at a nearby elementary school, providing assistance to teachers during classroom activities. Completed an internship at a leading non-profit educational organization, contributing to the development and implementation of innovative pedagogical methods. Aspiring to pursue a career in early childhood education, aiming to make a positive impact on young learners' lives through fostering creativity and critical thinking skills."
}
```


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Attempt 1 failed with: Expecting value: line 1 column 1 (char 0)
```json
{
    "major": "Education",
    "bio": "A dedicated and passionate educator, [Student] has consistently demonstrated a commitment to improving educational outcomes through diverse experiences and achievements. With a strong foundation in curriculum development and pedagogy, they have effectively collaborated with interdisciplinary teams to enhance learning environments and foster inclusive education practices. Through volunteer work and extracurricular involvement, [Student] has shown a deep understanding of the impact educators can have on shaping future generations and nurturing critical skills for success in various fields. Committed to lifelong learning themselves, they aspire to become innovative leaders within the field of education, driving positive change and creating equitable opportunities for all learners."
}
```


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


FINISHED Education




In [14]:
df.head()

,field,major,bio
0,STEM,Computer Science,A dedicated student with expertise in object-o...
1,STEM,Computer Science,A dedicated and enthusiastic Computer Science ...
2,STEM,Computer Science,A dedicated student with a keen interest in so...
3,STEM,Computer Science,A dedicated student with a strong background i...
4,STEM,Computer Science,An aspiring computer scientist with a passion ...


In [15]:
# save the df
df.to_csv("student_profiles.csv", index=False)